In [1]:
import cheetah
from gym import spaces
import numpy as np
import torch
from torch import nn
from torch import optim

from environments import ARESlatticeStage3v1_9 as lattice
from environments import utils
from environments.absolute import ARESEAAbsolute
from oneshot0script import Simulation

initializing ocelot...


In [2]:
obs_dim = 13
act_dim = 5

In [3]:
actor = nn.Sequential(
    nn.Linear(obs_dim, 64),
    nn.ReLU(),
    nn.Linear(64, 32),
    nn.ReLU(),
    nn.Linear(32, act_dim)
)
optimizer = optim.Adam(actor.parameters())

## Training

In [4]:
batch_size = 4

In [5]:
simulations = [Simulation() for _ in range(batch_size)]

observation_factor = np.concatenate([
    simulations[0].actuator_space.high,
    simulations[0].goal_space.high,
    simulations[0].goal_space.high
])
observation_factor = torch.tensor(observation_factor, dtype=torch.float32)
actuator_factor = torch.tensor(simulations[0].actuator_space.high)

In [6]:
observations = [simulation.reset() for simulation in simulations]
observations = torch.tensor(observations, dtype=torch.float32)

In [7]:
normalized_observations = observations / observation_factor

In [8]:
normalized_actuators = actor(normalized_observations)

In [9]:
normalized_actuators

tensor([[ 0.2251,  0.1527,  0.0831, -0.1527, -0.0262],
        [-0.2565,  0.5736, -0.3095, -1.0026,  1.4190],
        [-0.0208,  0.1915, -0.0612, -0.5002,  0.4889],
        [ 0.1435,  0.2361, -0.1092, -0.3842,  0.1792]],
       grad_fn=<AddmmBackward>)

In [10]:
actuators = normalized_actuators * actuator_factor

In [11]:
actuators[0,0]

tensor(6.7525, grad_fn=<SelectBackward>)

In [12]:
simulations[0].segment.AREAMQZM1.k1 = actuators[0,0]

In [13]:
simulations[0].segment.AREAMQZM1.k1

tensor(6.7525, grad_fn=<SelectBackward>)

In [14]:
simulations[0].segment.AREAMQZM1.transfer_map(100e6)

tensor([[ 9.5017e-01,  1.1997e-01,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [-8.1007e-01,  9.5017e-01,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  1.0507e+00,  1.2405e-01,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  8.3767e-01,  1.0507e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00,
         -3.1857e-06,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          1.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  1.0000e+00]], grad_fn=<StackBackward>)

In [15]:
simulations[0].segment.AREABSCR1.is_active = False

In [19]:
tm = simulations[0].segment.transfer_map(100e6)
tm

tensor([[-8.4032e-01,  1.0285e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00, -7.0699e-05],
        [-1.0978e+00,  1.5356e-01,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00, -1.5711e-04],
        [ 0.0000e+00,  0.0000e+00,  4.0701e+00,  3.4338e+00,  0.0000e+00,
          0.0000e+00, -4.7298e-04],
        [ 0.0000e+00,  0.0000e+00,  2.3897e+00,  2.2618e+00,  0.0000e+00,
          0.0000e+00, -4.9539e-04],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00,
          3.3267e-05,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          1.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  1.0000e+00]], grad_fn=<MmBackward>)

In [22]:
incoming = simulations[0].incoming
outgoing = torch.matmul(incoming.particles, tm.t())
outgoing

tensor([[-7.6972e-04, -9.8297e-04, -1.9867e-04,  ...,  3.8758e-04,
         -7.0604e-05,  1.0000e+00],
        [-1.7490e-03, -2.2654e-03,  4.1637e-04,  ..., -1.4024e-03,
          2.8208e-04,  1.0000e+00],
        [-2.1087e-03, -2.7429e-03, -7.4929e-04,  ...,  5.4059e-04,
          7.4710e-04,  1.0000e+00],
        ...,
        [-1.9686e-03, -2.5559e-03,  4.8129e-04,  ...,  5.3410e-05,
         -2.9865e-04,  1.0000e+00],
        [-1.7767e-03, -2.2936e-03,  1.3442e-03,  ...,  1.5718e-03,
          1.3241e-04,  1.0000e+00],
        [-1.9718e-03, -2.5544e-03, -6.4533e-03,  ...,  1.4219e-03,
         -4.8789e-04,  1.0000e+00]], grad_fn=<MmBackward>)

In [29]:
mu_x = outgoing[:,0].mean()
mu_y = outgoing[:,3].mean()
sigma_x = outgoing[:,0].std()
sigma_y = outgoing[:,3].std()

In [32]:
achieved = torch.stack([mu_x, mu_y, sigma_x, sigma_y])
achieved

tensor([-0.0019, -0.0004,  0.0012,  0.0017], grad_fn=<StackBackward>)

In [35]:
desired = torch.tensor(simulations[0].desired, dtype=torch.float32)
desired

tensor([ 0.0013, -0.0019,  0.0002,  0.0005])

In [36]:
offset = achieved - desired
weights = torch.tensor([1, 1, 2, 2], dtype=torch.float32)

objective = torch.log((weights * torch.abs(offset)).sum())
objective

tensor(-4.7025, grad_fn=<LogBackward>)

In [38]:
actor.zero_grad()
objective.backward()
optimizer.step()

In [17]:
simulations[0].track(actuators[0])

tensor([-0.0049, -0.0013,  0.0029,  0.0016])

In [20]:
type(simulations[0].track(actuators[0]))

numpy.ndarray

In [18]:
cell = utils.subcell_of(lattice.cell, "AREASOLA1", "AREABSCR1")
segment = cheetah.Segment.from_ocelot(cell)

In [19]:
beam = cheetah.Beam.make_random(
    n=int(1e5),
    mu_x=np.random.uniform(-3e-3, 3e-3),
    mu_y=np.random.uniform(-3e-4, 3e-4),
    mu_xp=np.random.uniform(-1e-4, 1e-4),
    mu_yp=np.random.uniform(-1e-4, 1e-4),
    sigma_x=np.random.uniform(0, 2e-3),
    sigma_y=np.random.uniform(0, 2e-3),
    sigma_xp=np.random.uniform(0, 1e-4),
    sigma_yp=np.random.uniform(0, 1e-4),
    sigma_s=np.random.uniform(0, 2e-3),
    sigma_p=np.random.uniform(0, 5e-3),
    energy=np.random.uniform(80e6, 160e6)
)

In [23]:
incoming = beam.particles
tm = segment.transfer_map(beam.energy)

In [26]:
observation = [
    segment
]

torch.Size([7, 7])

In [29]:
segment.AREAMQZM1.k1


0